# Transfer Learningのお勉強

*20170621 Mitsuhiro Ito*

自分で書いてみよう！  
目標　高精度な犬猫を分類するモデルの構築

* [keras公式ブログ](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
* [keras公式ドキュメント](https://keras.io/applications)
* [@aidiaryさんの記事](http://aidiary.hatenablog.com/entry/20170110/1484057655)

---

# 1. 前準備

In [85]:
from __future__ import print_function, division
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [39]:
import sys
sys.path.append('/Users/mitsuhiro/Work/intro_machine_learning')
from utils import save_model_viz, save_weights, save_hist, plot_hist
from utils import load_mnist_3D

# 2. 画像ファイルの入手


犬猫データはKaggleからゲットしました  
登録がめんどくさいです  
https://www.kaggle.com/c/dogs-vs-cats/data


次に画像ファイルをtrain用1000枚、validation用400枚に分けます

In [68]:
import os

In [87]:
source_dir = "/Users/mitsuhiro/Work/train"　#画像ファイルへのpath

In [89]:
os.makedirs("%s/data" % "/Users/mitsuhiro/Work")　#train、valid用のフォルダを順次作成
data_dir = "/Users/mitsuhiro/Work/data"

In [90]:
os.makedirs("%s/train" % data_dir)
os.makedirs("%s/valid" % data_dir)
train_dir = "/Users/mitsuhiro/Work/data/train"
valid_dir = "/Users/mitsuhiro/Work/data/valid"

In [91]:
os.makedirs("%s/dogs" % train_dir)
os.makedirs("%s/cats" % train_dir)
os.makedirs("%s/dogs" % valid_dir)
os.makedirs("%s/cats" % valid_dir)

In [92]:
for i in range(1000):　#train用画像ファイルを移動
    os.rename("%s/dog.%d.jpg" % (source_dir, i + 1),
              "%s/dogs/dog%04d.jpg" % (train_dir, i + 1))
    os.rename("%s/cat.%d.jpg" % (source_dir, i + 1),
              "%s/cats/cat%04d.jpg" % (train_dir, i + 1))

In [93]:
ls "/Users/mitsuhiro/Work/data/train/cats"

cat0001.jpg  cat0168.jpg  cat0335.jpg  cat0502.jpg  cat0669.jpg  cat0836.jpg
cat0002.jpg  cat0169.jpg  cat0336.jpg  cat0503.jpg  cat0670.jpg  cat0837.jpg
cat0003.jpg  cat0170.jpg  cat0337.jpg  cat0504.jpg  cat0671.jpg  cat0838.jpg
cat0004.jpg  cat0171.jpg  cat0338.jpg  cat0505.jpg  cat0672.jpg  cat0839.jpg
cat0005.jpg  cat0172.jpg  cat0339.jpg  cat0506.jpg  cat0673.jpg  cat0840.jpg
cat0006.jpg  cat0173.jpg  cat0340.jpg  cat0507.jpg  cat0674.jpg  cat0841.jpg
cat0007.jpg  cat0174.jpg  cat0341.jpg  cat0508.jpg  cat0675.jpg  cat0842.jpg
cat0008.jpg  cat0175.jpg  cat0342.jpg  cat0509.jpg  cat0676.jpg  cat0843.jpg
cat0009.jpg  cat0176.jpg  cat0343.jpg  cat0510.jpg  cat0677.jpg  cat0844.jpg
cat0010.jpg  cat0177.jpg  cat0344.jpg  cat0511.jpg  cat0678.jpg  cat0845.jpg
cat0011.jpg  cat0178.jpg  cat0345.jpg  cat0512.jpg  cat0679.jpg  cat0846.jpg
cat0012.jpg  cat0179.jpg  cat0346.jpg  cat0513.jpg  cat0680.jpg  cat0847.jpg
cat0013.jpg  cat0180.jpg  cat0347.jpg  cat0514.jpg  cat0681.jpg  cat0848.jpg

In [95]:
for i in range(400):#valid用画像ファイルを移動
    os.rename("%s/dog.%d.jpg" % (source_dir, 1000 + i + 1),
              "%s/dogs/dog%04d.jpg" % (valid_dir, i + 1))
    os.rename("%s/cat.%d.jpg" % (source_dir, 1000 + i + 1),
              "%s/cats/cat%04d.jpg" % (valid_dir, i + 1))


こんな感じになりました

    data/
        train/
              dogs/
                  dog.0001.jpg  ...  dog.1000.jpg
              cats/
                  cat.0001.jpg  ...  cat.1000.jp
                
        valid/
              dogs/
                  dog.0001.jpg  ...  dog.0400.jpg
              cats/
                　cat.0001.jpg  ...  cat.0400.jpg
                
---

# 3. 画像データへ加工

In [60]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [61]:
generator = ImageDataGenerator(rescale=1.0 / 255) 

ImageDataGeneratorによりリアルタイムに，画像データを生成する  
引数にはいろいろあるがrescaleなしで4-1.のCNNをやった場合、val_actが0.5付近から全然増えない...

ImageDataGenerator参考
https://keras.io/ja/preprocessing/image/


In [62]:
train_generator = generator.flow_from_directory('/Users/mitsuhiro/Work/data/train/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')
valid_generator = generator.flow_from_directory('/Users/mitsuhiro/Work/data/valid/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


このgeneratorに渡せばデータ化してくれる

---




# 4.1 モデル構築＆評価　CNN

In [63]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [64]:
RUN_ID = 'CNN'

In [65]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [66]:
save_model_viz(RUN_ID, model)

In [67]:
batch_size = 16
hist = model.fit_generator(train_generator, 
                    steps_per_epoch = 2000 // batch_size, 
                    epochs=10, 
                    verbose=1, 
                    validation_data = valid_generator,
                    validation_steps= 800 // batch_size)

save_weights(RUN_ID, model)
save_hist(RUN_ID, hist)

Epoch 1/10
125/125 [==============================] - 135s - loss: 0.6842 - acc: 0.5537 - val_loss: 0.6773 - val_acc: 0.6262
Epoch 2/10
125/125 [==============================] - 132s - loss: 0.6217 - acc: 0.6612 - val_loss: 0.6618 - val_acc: 0.6112
Epoch 3/10
125/125 [==============================] - 131s - loss: 0.5487 - acc: 0.7205 - val_loss: 0.5782 - val_acc: 0.6969
Epoch 4/10
125/125 [==============================] - 126s - loss: 0.4770 - acc: 0.7750 - val_loss: 0.5577 - val_acc: 0.7181
Epoch 5/10
125/125 [==============================] - 132s - loss: 0.3877 - acc: 0.8265 - val_loss: 0.6503 - val_acc: 0.6806
Epoch 6/10
125/125 [==============================] - 132s - loss: 0.3077 - acc: 0.8673 - val_loss: 0.6610 - val_acc: 0.7219
Epoch 7/10
125/125 [==============================] - 136s - loss: 0.2258 - acc: 0.9057 - val_loss: 0.7964 - val_acc: 0.7306
Epoch 8/10
125/125 [==============================] - 138s - loss: 0.1489 - acc: 0.9425 - val_loss: 0.8311 - val_acc: 0.7381


In [54]:
save_weights(RUN_ID, model)
save_hist(RUN_ID, hist)

In [69]:
plot_hist(RUN_ID)　　### 図でない　力宗さんにきく

In [70]:
from IPython.display import Image
Image("model/mlp_vis.png")　### 図でない　力宗さんにきく

---

# 4.2 モデル構築＆評価　VGG16を特長抽出機として使用

In [71]:
from keras.applications.vgg16 import VGG16

In [72]:
model = VGG16(weights='imagenet')

Exception: URL fetch failure on https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5: None -- [Errno 60] Operation timed out

In [ ]:
bottleneck_features_train = model.predict_generator(generator, nb_train_samples)
np.save(os.path.join(result_dir, 'bottleneck_features_train.npy'),　bottleneck_features_train)

In [ ]:
model.predict_generator()

In [ ]:
predict_generator(self, generator, val_samples, max_q_size=10, nb_worker=1, pickle_safe=False)

In [67]:
os.chdir("/Users/mitsuhiro/Work")

---
os.renameの使い方

In [56]:
os.chdir("/Users/mitsuhiro/Desktop")

In [57]:
ls 

D論発表18.pptx*                 confidentialのエイリアス
Workのエイリアス                研究発表1.pptx*
cat.14.jpg


In [58]:
os.rename("%s/cat.14.jpg" % "/Users/mitsuhiro/Desktop" , "%s/cat.14.jpg" % "/Users/mitsuhiro/work") #%の後部分が%sに入る